In [1]:
import sys,os,errno,signal,copy
from contextlib import contextmanager
# !pip install intervaltree
import numpy as np
import musicnet

import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.functional import conv1d, mse_loss, cross_entropy
print(torch.__version__)
from time import time

import matplotlib.pyplot as plt
from IPython.display import Audio

from sklearn.metrics import average_precision_score

root = '/mnt/musicnet/'
checkpoint_path = './checkpoints'
checkpoint = 'musicnet_demo.pt'

try:
    os.makedirs(checkpoint_path)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

%matplotlib inline

0.3.1


In [2]:
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'   # see issue #152
os.environ['CUDA_VISIBLE_DEVICES']='1,2,3'

def worker_init(args):
    signal.signal(signal.SIGINT, signal.SIG_IGN) # ignore signals so parent can handle them
    np.random.seed(os.getpid() ^ int(time())) # approximately random seed for workers

batch_size = 100
kwargs = {'num_workers': 4, 'pin_memory': True, 'worker_init_fn': worker_init}

m = 128
k = 256
d = 4096
window = 16384
stride = 512
regions = 1 + (window - d)/stride
# sig_len = 

In [3]:
train_set = musicnet.MusicNet(root=root, train=True, download=False, window=window)#, pitch_shift=5, jitter=.1)
test_set = musicnet.MusicNet(root=root, train=False, window=window, epoch_size=50000)

In [4]:
naruto_test = musicnet.MusicNet(root=root, train=False, window=window, download=False, epoch_size=50000, naruto=True)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,**kwargs)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,**kwargs)
naruto_loader = torch.utils.data.DataLoader(dataset=naruto_test,batch_size=batch_size,**kwargs)

In [6]:
def create_filters(d,k,low=50,high=6000):
    x = np.linspace(0, 2*np.pi, d, endpoint=False)
    wsin = np.empty((k,1,d), dtype=np.float32)
    wcos = np.empty((k,1,d), dtype=np.float32)
    start_freq = low
    end_freq = high
    num_cycles = start_freq*d/44000.
    scaling_ind = np.log(end_freq/start_freq)/k
    window_mask = 1.0-1.0*np.cos(x)
    for ind in range(k):
        wsin[ind,0,:] = window_mask*np.sin(np.exp(ind*scaling_ind)*num_cycles*x)
        wcos[ind,0,:] = window_mask*np.cos(np.exp(ind*scaling_ind)*num_cycles*x)
    
    return wsin,wcos

In [7]:
def L(y_hat, y):
    # adjust for per-frame loss
    return mse_loss(y_hat, y)*128/2.

def L(y_hat, y):
    # adjust for per-frame loss
    return cross_entropy(y_hat, y.type(torch.cuda.LongTensor))

In [8]:
class Flatten(torch.nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [9]:
class Model(torch.nn.Module):
    def __init__(self, avg=.9998):
        super(Model, self).__init__()
        
        wsin,wcos = create_filters(d,k)
        with torch.cuda.device(0):
            self.wsin_var = Variable(torch.from_numpy(wsin).cuda(), requires_grad=False)
            self.wcos_var = Variable(torch.from_numpy(wcos).cuda(), requires_grad=False)
        h1 = 256
        h2 = 256
        self.seq = torch.nn.Sequential()
        conv1_stride = 1
        self.seq.add_module('drop', torch.nn.Dropout())
#         self.seq.add_module('bn', torch.nn.BatchNorm1d(k))
        
        self.seq.add_module('conv1', torch.nn.Conv1d(1, 16, kernel_size = 16, padding=7, stride=16))
        self.seq.add_module('conv2', torch.nn.Conv1d(16, 64, kernel_size = 16, padding=7, stride=16))
#         self.seq.add_module('conv1', torch.nn.Conv1d(1, 64, kernel_size = 3, padding=1, stride=1))
#         self.seq.add_module('conv1', torch.nn.Conv1d(1, 64, kernel_size = 3, padding=1, stride=1))

        self.seq.add_module('flat', Flatten())
#         self.seq.add_module('conv2', torch.nn.Conv1d(64, 16, kernel_size = 3, padding=1, stride=1))
#         self.seq.add_module('conv3', torch.nn.Conv1d(16, 1, kernel_size = 3, padding=1, stride=1))
        self.seq.add_module('lout', torch.nn.Linear(int(window/16/16*64), int(m), bias=False))
        
#         self.seq.add_module('l1', torch.nn.Linear(int(regions*k), int(h1), bias=False))
#         self.seq.add_module('relu1', torch.nn.ELU())
#         self.seq.add_module('l2', torch.nn.Linear(int(h1), int(h2), bias=False))
#         self.seq.add_module('relu2', torch.nn.ELU())
#         self.seq.add_module('l3', torch.nn.Linear(int(h2), int(m), bias=False))


        self.seq.add_module('out', torch.nn.LogSoftmax())
    
    
#         self.seq.add_module('out', torch.nn.LogSigmoid())
#         self.seq.add_module('out', torch.nn.Sigmoid())
#         self.linear = torch.nn.Linear(int(regions*k), int(h), bias=False).cuda()
#         torch.nn.init.xavier_uniform(self.linear.weight)
#         self.linear2 = torch.nn.Linear(int(h), int(h), bias=False).cuda()
#         torch.nn.init.xavier_uniform(self.linear2.weight)
#         self.linear3 = torch.nn.Linear(int(h), int(m), bias=False).cuda()
#         torch.nn.init.xavier_uniform(self.linear3.weight)
        self.seq.cuda()
        self.avg = avg
        self.averages = copy.deepcopy(list(parm.data for parm in self.parameters()))
        for (name,parm),pavg in zip(self.named_parameters(),self.averages):
            self.register_buffer(name + '.avg', pavg)
    
    def forward(self, x):
#         print(x.size())
        zx = conv1d(x[:,None,:], self.wsin_var, stride=stride).pow(2) \
           + conv1d(x[:,None,:], self.wcos_var, stride=stride).pow(2)
#         print(zx.size())
#         hid = self.linear(torch.log(zx + musicnet.epsilon).view(x.data.size()[0],int(regions*k)))
#         hid2 = self.linear2(F.relu(hid))
#         hid3 = self.linear3(F.relu(hid2))
#         return F.softmax(hid3)
#         return self.seq.forward(torch.log(zx + musicnet.epsilon).view(x.data.size()[0],int(regions*k)))
#         return self.seq.forward(torch.log(zx + musicnet.epsilon))print
        return self.seq.forward(x.view(x.data.size()[0],1,int(window)))

    
    def average_iterates(self):
        for parm, pavg in zip(self.parameters(), self.averages):
            pavg.mul_(self.avg).add_(1.-self.avg, parm.data)

@contextmanager
def averages(model):
    orig_parms = copy.deepcopy(list(parm.data for parm in model.parameters()))
    for parm, pavg in zip(model.parameters(), model.averages):
        parm.data.copy_(pavg)
    yield
    for parm, orig in zip(model.parameters(), orig_parms):
        parm.data.copy_(orig)

# Train the model

In [10]:
model = Model()
print (model)
loss_history = []
avgp_history = []

Model(
  (seq): Sequential(
    (drop): Dropout(p=0.5)
    (conv1): Conv1d(1, 16, kernel_size=(16,), stride=(16,), padding=(7,))
    (conv2): Conv1d(16, 64, kernel_size=(16,), stride=(16,), padding=(7,))
    (flat): Flatten(
    )
    (lout): Linear(in_features=4096, out_features=128, bias=False)
    (out): LogSoftmax()
  )
)


In [ ]:
# (optional) restore from checkpoint, if it exists
# try:
#     model.load_state_dict(torch.load(os.path.join(checkpoint_path,checkpoint)))
# except IOError as e:
#     if e.errno != errno.ENOENT:
#         raise

In [ ]:
# optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=.95)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.MultiLabelSoftMarginLoss()
try:
    with train_set, test_set:
        print ('square loss\tavg prec\ttime\t\tutime')
        for epoch in range(500):
            t = time()
            for i, (x, y) in enumerate(train_loader):
                x, y = Variable(x.cuda(), requires_grad=True), Variable(y.cuda(), requires_grad=False)
#                 print(y.mean())
                #                 loss = L(model(x),y)
                loss = criterion(model(x),y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                model.average_iterates()

            t1 = time()
            avgp, loss = 0., 0.
            yground = torch.FloatTensor(batch_size*len(test_loader), m)
            yhat = torch.FloatTensor(batch_size*len(test_loader), m)
            with averages(model):
                for i, (x, y) in enumerate(test_loader):
                    x, y = Variable(x.cuda(), requires_grad=False), Variable(y.cuda(), requires_grad=False)
                    yhatvar = model(x)
#                     loss += L(yhatvar,y).data[0]
                    loss += criterion(yhatvar,y).data[0]
                    yground[i*batch_size:(i+1)*batch_size,:] = y.data
                    yhat[i*batch_size:(i+1)*batch_size,:] = yhatvar.data
            avgp = average_precision_score(yground.numpy().flatten(),yhat.numpy().flatten())
            loss_history.append(loss/len(test_loader))
            avgp_history.append(avgp)
            torch.save(model.state_dict(), os.path.join(checkpoint_path,checkpoint))
            print ('{:2f}\t{:2f}\t{:2f}\t{:2f}'.format(loss_history[-1],avgp_history[-1],time()-t, time()-t1))

except KeyboardInterrupt:
    print ('Graceful Exit')
else:
    print ('Finished')

square loss	avg prec	time		utime


/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.089055	0.058829	28.694576	8.882244
0.083846	0.064142	27.502878	9.146464
0.080853	0.066408	26.665220	9.206639
0.078249	0.066653	25.998415	9.584826
0.077299	0.066661	25.144927	9.498252
0.077217	0.067249	26.190445	9.583795
0.076527	0.067736	25.122895	9.943019
0.076456	0.067106	25.297190	10.193533
0.076178	0.066996	25.038112	9.296016
0.076028	0.067802	25.877631	8.682629
0.076054	0.066966	27.029356	9.661135
0.076183	0.067389	27.316587	9.748882
0.076029	0.067684	27.500017	9.950887
0.076082	0.068249	27.004697	9.442557
0.076331	0.067837	28.146962	11.995288
0.076631	0.068431	28.311491	10.973042
0.076420	0.068364	25.732841	9.581273
0.076318	0.068023	25.655419	9.768300
0.075830	0.067691	25.331743	9.688355
0.076058	0.068403	25.584362	10.205876
0.075961	0.068765	25.606829	9.786461
0.076408	0.068780	25.482717	9.685436
0.076178	0.067741	22.233274	9.080993
0.076104	0.068731	26.012631	9.297248
0.076316	0.069025	26.101165	8.692589
0.076220	0.068548	26.277414	9.029017
0.076016	0.069684	23.243008	8.2332

In [ ]:
print(batch_size*len(train_loader))
print(batch_size*len(test_loader))

In [ ]:
burnin = 1

fig = plt.figure(figsize=(15, 7))
fig.add_axes()

ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)

for ax in [ax1, ax2]:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.grid(color='b', linestyle='--', linewidth=0.5, alpha=0.3)
    ax.tick_params(direction='out', color='b', width='2')
    
ax1.set_title('square loss')
ax1.plot(np.arange(len(loss_history[burnin:])), loss_history[burnin:])
ax2.set_title('average precision')
ax2.plot(np.arange(len(avgp_history[burnin:])), avgp_history[burnin:])

# Реал тест на наруте

In [ ]:

try:
    with train_set, test_set:
        print ('square loss\tavg prec\ttime\t\tutime')
        for epoch in range(50):
            t = time()
            for x, y in train_loader:
                x, y = Variable(x.cuda(), requires_grad=False), Variable(y.cuda(), requires_grad=False)
                loss = L(model(x),y)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                model.average_iterates()

            t1 = time()
            avgp, loss = 0., 0.
            yground = torch.FloatTensor(batch_size*len(test_loader), m)
            yhat = torch.FloatTensor(batch_size*len(test_loader), m)
            with averages(model):
                for i, (x, y) in enumerate(naruto_loader):
                    x, y = Variable(x.cuda(), requires_grad=False), Variable(y.cuda(), requires_grad=False)
                    yhatvar = model(x)
                    print(yhatvar)
                    loss += L(yhatvar,y).data[0]
                    yground[i*batch_size:(i+1)*batch_size,:] = y.data
                    yhat[i*batch_size:(i+1)*batch_size,:] = yhatvar.data
            avgp = average_precision_score(yground.numpy().flatten(),yhat.numpy().flatten())
            loss_history.append(loss/len(test_loader))
            avgp_history.append(avgp)
            torch.save(model.state_dict(), os.path.join(checkpoint_path,checkpoint))
            print ('{:2f}\t{:2f}\t{:2f}\t{:2f}'.format(loss_history[-1],avgp_history[-1],time()-t, time()-t1))

except KeyboardInterrupt:
    print ('Graceful Exit')
else:
    print ('Finished')